<a href="https://colab.research.google.com/github/mysend12/algorithm/blob/main/3%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 협업 필터링 추천 시스템

어떤 아이템에 대해 비슷한 취향을 가진 사람들은 다른 아이템 또한 비슷한 취향을 가질 것이다.  
협업 필터링(Collaborative Filtering: CF)

## 원리
취향이 비슷한 사람들의 집단 존재한다고 가정  
추천 대상의 유사 집단을 찾아서 그들이 좋아하는 아이템을 추천  

CF에서는 사용자간의 유사도를 찾는것이 핵심

## 유사도 지표

### 상관계수
- 가장 이해하기 쉬운 유사도
- -1 ~ 1 사이의 값  
-- 1인 경우 양의 상관관계
-- -1에 가까우면 음의 상관관계
- 0인 경우 상관관계가 없거나, U 형 상관관계 등 일반적이지 않은 상관관계

### 코사인 유사도
1. 협업 필터링에서 가장 널리 쓰이는 유사도
2. 각 아이템 => 하나의 차원, 사용자의 평가값 => 좌표값
3. 두 사용자의 평가값 유사 => theta는 작아지고, 코사인 값은 커짐
4. -1 ~ 1 사이의 값
5. 데이터 이진값(binary) => 타니모토 계수(tanimoto coefficient) 사용 권장


### 자카드 계수
1. 타니모토 계수의 변형 => 자카드 계수
2. 이진수 데이터 => 좋은 결과


## 기본 CF 알고리즘
1. 모든 사용자 간 평가의 유사도 계산
2. 추천 대상과 다른 사용자간 유사도 추출
3. 추천 대상이 평가하지 않은 아이템에 대한 예상 평가값 계산
- 평가값 = 다른 사용자 평가 * 다른 사용자 유사도
4. 아이템 중 예상 평가값이 가장 높은 N개 추천

In [1]:
# 사용자 u.user 파일을 DataFrame으로 읽기

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

##### 데이터 불러오기 및 필요한 함수 정의 #####
base_src = 'drive/MyDrive/RecoSys/Data'

# user 데이터

# user
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    u_user_src,
    sep='|',
    names=u_cols,
    encoding='latin-1'
)
users = users.set_index('user_id')

# movie
u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id', 'title', 'release data', 'video release data', 'IMDB URL', 'unknown', 'Action', 'Adventure', 'Animation',
          'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
          'Thriller', 'War', 'Western'
          ]
movies = pd.read_csv(u_item_src, sep='|', names=i_cols, encoding='latin-1')
movies = movies.set_index('movie_id')

# rating
u_data_src = os.path.join(base_src, 'u.data')
u_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(
    u_data_src,
    sep='\t',
    names=u_cols,
    encoding='latin-1'
)

def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# score(RMSE) 계산
def score(model):
  # 테스트 데이터의 user_id와 movie_id 간 pair를 맞춰 튜플형 원소 리스트 데이터를 만든다.
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  # 모든 사용자-영화 짝에 대해서 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성
  y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
  # 실제 평점값
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

##### 데이터 셋 만들기 #####
x = ratings.copy()
y= ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)
ratings_matrix = x_train.pivot(index='user_id', columns = 'movie_id', values='rating')

##### 코사인 유사도 계산 #####
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = ratings_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=ratings_matrix.index, columns=ratings_matrix.index)

##### 주어진 영화의(movie_id) 가중평균 rating을 계산하는 함수 #####
def CF_simple(user_id, movie_id):
  ## 지정한 movie_id가 rating metrix의 column 안에 있는 경우
  if movie_id in ratings_matrix.columns:
    ## 주어진 사용자와 다른 사용자간의 유사성을 추출하여 복제
    sim_scores = user_similarity[user_id].copy()
    ## 주어진 영화의 다른 사용자들에 의한 평가
    movie_ratings = ratings_matrix[movie_id].copy()
    ## 주어진 영화에 대해서 평가를 하지 않은 사용자의 인덱스를 추출
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    ## 평가하지 않은 사용자들을 영화 평점에서 제외
    movie_ratings = movie_ratings.dropna()
    ## 평가하지 않은 사용자들을 유사도에서도 제외
    sim_scores = sim_scores.drop(none_rating_idx)
    ## 평가한 사람들을 가중평균하여 나누기
    mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
  else:
    mean_rating = 3.0
  return mean_rating

##### 정확도 계산 #####
score(CF_simple)



1.024934092316392

# 이웃을 고려한 CF

모든 사용자들간의 유사도를 가중평균하는것이 아니라, 유사도가 높은 사람들간의 유사도를 가중평균

개선 방법
1. KNN 방법
2. Thresholding 방법

In [33]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

##### 데이터 불러오기 및 필요한 함수 정의 #####
base_src = 'drive/MyDrive/RecoSys/Data'

# user 데이터

# user
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    u_user_src,
    sep='|',
    names=u_cols,
    encoding='latin-1'
)
users = users.set_index('user_id')

# movie
u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id', 'title', 'release data', 'video release data', 'IMDB URL', 'unknown', 'Action', 'Adventure', 'Animation',
          'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
          'Thriller', 'War', 'Western'
          ]
movies = pd.read_csv(u_item_src, sep='|', names=i_cols, encoding='latin-1')
movies = movies.set_index('movie_id')

# rating
u_data_src = os.path.join(base_src, 'u.data')
u_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(
    u_data_src,
    sep='\t',
    names=u_cols,
    encoding='latin-1'
)

def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# 유사집단의 크기를 미리 정하기 위해서 기존 score 함수에 neighbor_size 인자값 추가
def score(model, neighbor_size=0):
  # 테스트 데이터의 user_id와 movie_id 간 pair를 맞춰 튜플형 원소 리스트 데이터를 만든다.
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  # 모든 사용자-영화 짝에 대해서 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성
  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
  # 실제 평점값
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

##### 데이터 셋 만들기 #####
x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)
ratings_matrix = x_train.pivot(index='user_id', columns = 'movie_id', values='rating')

##### train set의 모든 가능한 사용자 pair의 cosine similarities 계산 #####
# 코사인 유사도 계산 #
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = ratings_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=ratings_matrix.index, columns=ratings_matrix.index)

##### 주어진 영화의(movie_id) 가중평균 rating을 계산하는 함수 #####
def CF_simple(user_id, movie_id):
  ## 지정한 movie_id가 rating metrix의 column 안에 있는 경우
  if movie_id in ratings_matrix.columns:
    ## 주어진 사용자와 다른 사용자간의 유사성을 추출하여 복제
    sim_scores = user_similarity[user_id].copy()
    ## 주어진 영화의 다른 사용자들에 의한 평가
    movie_ratings = ratings_matrix[movie_id].copy()
    ## 주어진 영화에 대해서 평가를 하지 않은 사용자의 인덱스를 추출
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    ## 평가하지 않은 사용자들을 영화 평점에서 제외
    movie_ratings = movie_ratings.dropna()
    ## 평가하지 않은 사용자들을 유사도에서도 제외
    sim_scores = sim_scores.drop(none_rating_idx)
    ## 평가한 사람들을 가중평균하여 나누기
    mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
  else:
    mean_rating = 3.0
  return mean_rating

##### Neighbor size를 정해서 예측치를 계산하는 함수 #####
def CF_knn(user_id, movie_id, neighbor_size=0):
  if movie_id in ratings_matrix.columns:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = ratings_matrix[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)

    if neighbor_size == 0:
      mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    else:
      if len(sim_scores) > 1:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_socres = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        # user_idx 기반으로 sim_scores를 정렬해서 -neighbor_size만큼 뒤에서부터(정렬 기준에 따라 다름) 뽑아낸다.
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      else:
        mean_rating = 3.0
  else:
    mean_rating = 3.0

  return mean_rating

# 정확도 계산
score(CF_knn, neighbor_size=30)
# user_similarity[729].copy()


KeyError: '[126, 20, 70, 93, 15, 73, 28, 103, 29, 118, 17, 50, 61, 54, 69, 94, 3, 38, 19, 111, 97, 8, 98, 143, 124, 84, 27, 131, 66, 79, 39, 16, 120, 135, 53, 117, 108, 32, 47, 71, 63, 35, 44, 0, 24, 56, 102, 87, 89, 137, 4, 125, 110, 51, 36, 34, 78, 77, 95, 2, 116, 18, 101, 48, 43, 74, 55, 45, 115, 86, 136, 128, 67, 133, 134, 52, 129, 68, 76, 139, 58, 122, 123, 7, 83, 40, 121, 112, 64, 91, 41, 60, 88, 23, 107, 65, 90, 42, 12, 22, 33, 21, 72, 100, 31, 6, 80, 10, 14, 140, 113, 25, 11, 105, 5, 109, 46, 85, 57, 26, 30, 99, 9, 127, 96, 132, 114, 106] not in index'

In [15]:
##### 실제 주어진 사용자에 대해 추천을 받는 기능 구현 #####
ratings_matrix = ratings.pivot_table(
    index='user_id', columns = 'movie_id', values='rating')

matrix_dummy = ratings_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=ratings_matrix.index, columns=ratings_matrix.index)

# n_items: 몇 개 추천
def recom_movie(user_id, n_items, neighbor_size=30):
  user_movie = ratings_matrix.loc[user_id].copy()

  for movie in ratings_matrix.columns:
    if pd.notnull(user_movie.loc[movie]):
      user_movie.loc[movie] = 0

    else:
      user_movie.loc[movie] = CF_knn(user_id, movie, neighbor_size)

  # 예상 평점 기준으로 영화 개수 기준으로 추천
  movie_sort = user_movie.sort_values(ascending=False)[:n_items]
  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']
  return recommendations

recom_movie(user_id=729, n_items=5, neighbor_size=30)




KeyError: '[225, 304, 270, 32, 269, 34, 310, 37, 255, 163, 169, 175, 48, 258, 155, 446, 443, 436, 91, 408, 9, 354, 353, 352, 196, 74, 12, 132, 104, 237, 215, 302, 86, 364, 259, 0, 110, 100, 267, 39, 391, 229, 164, 76, 183, 272, 197, 240, 27, 426, 107, 439, 133, 161, 36, 51, 351, 217, 418, 118, 420, 190, 71, 180, 127, 192, 337, 116, 221, 206, 4, 421, 53, 105, 146, 154, 341, 386, 273, 47, 126, 367, 8, 3, 87, 334, 224, 433, 377, 370, 383, 195, 355, 376, 30, 140, 46, 266, 400, 40, 172, 186, 7, 214, 427, 404, 208, 143, 55, 19, 414, 238, 176, 369, 239, 114, 103, 405, 142, 260, 78, 156, 24, 428, 288, 173, 90, 430, 61, 111, 284, 335, 413, 444, 227, 211, 33, 122, 385, 315, 170, 165, 219, 35, 356, 60, 147, 281, 179, 362, 415, 191, 129, 149, 442, 11, 448, 451, 135, 152, 373, 409, 358, 323, 207, 283, 29, 316, 241, 375, 166, 68, 236, 88, 449, 220, 98, 139, 123, 226, 113, 328, 285, 257, 309, 31, 282, 228, 384, 278, 187, 372, 321, 167, 115, 119, 212, 112, 171, 205, 440, 264, 382, 317, 80, 233, 69, 34